In [1]:
import glob
import numpy as np
import pandas as pd

Firstly, we create a function to compare the columns of two csv's to drop columns which are not mutual. We will merge the daily csv files into quarterly csv files.

In [ ]:
def compare_df_columns_drop(csv_1, csv_2):
    "Function to compare two columns of a dataframe and drop the columns which are not mutual"
    # Load dataframes
    dataframe_1 = pd.read_csv(csv_1)
    dataframe_2 = pd.read_csv(csv_2)
    
    # Compare and drop unmutual oclumns
    unmutual_columns_df1 = []
    unmutual_columns_df2 = []
    
    # If dataframe 1 has different columns than dataframe 2, drop columns in dataframe 1
    for x in dataframe_1.columns.values:
        if x not in dataframe_2.columns.values:
            unmutual_columns_df1.append(x)
            if len(unmutual_columns_df1) > 0:
                dataframe_1 = dataframe_1.drop([str(x)], axis = 1)
                
    # If dataframe 2 has different columns than dataframe 1, drop columns in dataframe 2
    for x in dataframe_2.columns.values:
        if x not in dataframe_1.columns.values:
            unmutual_columns_df2.append(x)
            if len(unmutual_columns_df2) > 0:
                dataframe_2 = dataframe_2.drop([str(x)], axis = 1)
        
    return dataframe_1, dataframe_2   

In [ ]:
df_1, df_2 = compare_df_columns_drop('data/drive_stats_2019_Q1/2019-01-01.csv',
                        'data/data_Q4_2019/2019-10-01.csv') 

Next, we create a function to merge all the daily csv's files into quarterly csv files and then merging it all.

In [ ]:
def merged_csv(folder_name, output_name):
    "Function to merge all csv files in a folder"

    # Create empty dataframe with all columns
    tmp_df = pd.read_csv('data/drive_stats_2019_Q1/2019-01-01.csv')
    columns_df = tmp_df.columns.values
    
    # Create empty dataframe
    df = pd.DataFrame(columns = columns_df )
    
    # Merge CSV's
    folder_name = folder_name
    file_type = 'csv'
    
    unmutual_columns = []
    
    # Initiliaze all files in the folder
    for f in glob.glob(folder_name + "/*." + file_type):
        
        # If it's the last file in the folder, it will append all entries.
        if f == glob.glob(folder_name + "/*." + file_type)[-1]:
            tmp_df = pd.read_csv(f)
            
            # Drop columns that are unmutual
            for x in tmp_df.columns.values:
                if x not in df.columns.values:
                    unmutual_columns.append(x)
                    if len(unmutual_columns) > 0:
                        tmp_df = tmp_df.drop([str(x)], axis = 1)    
                        
            df = df.append(tmp_df) 
            
        # If not the last file in the folder, append all entires that failed.
        else:
            tmp_df = pd.read_csv(f)
            df = df.append(tmp_df[tmp_df['failure'] == 1])
    
    output_name = output_name
    df.to_csv(output_name, index = None, header = True)

In [ ]:
# Create quarterly files
merged_csv('data/drive_stats_2019_Q1', 'data/Q1-2019.csv')
merged_csv('data/data_Q2_2019', 'data/Q2-2019.csv')
merged_csv('data/data_Q3_2019', 'data/Q3-2019.csv')
merged_csv('data/data_Q4_2019', 'data/Q4-2019.csv')
merged_csv('data/','data/Full-2019.csv')

Finally, we create a new csv file to track the number of failures per quarter to track the frequency of failures. 

In [ ]:
def daily_hd_failures(directories):
    "Function which goes through various directories to record the number of hard drives and failures per day amd outputs a csv"
    date_list = []
    hard_drives_per_date = []
    failures_per_date = []
    # Loop through all individual date csv's and record the data
    for folder_name in directories:
        for f in glob.glob(folder_name + "/*." + 'csv'):
            ## Read in dataframe
            tmp_df = pd.read_csv(f)

            ## Record the date list, the number of hard drives and failures per date. 
            date_list.append(tmp_df['date'].unique()[0])
            hard_drives_per_date.append(tmp_df.shape[0])
            failures_per_date.append(len(tmp_df[tmp_df['failure'] == 1]))
            
    # Create dataframe of recorded data        
    data =  {'date': date_list,
             'hard_drives': hard_drives_per_date,
             'failures': failures_per_date,
            }

    df = pd.DataFrame(data)
            
    return df.to_csv('data/daily_hd_failures.csv', index = None, header = True)   

In [ ]:
daily_hd_failures(['data/drive_stats_2019_Q1', 'data/data_Q2_2019',
                        'data/data_Q3_2019', 'data/data_Q4_2019' ] 